In [5]:
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import joblib
import sys

In [6]:
import json

In [7]:
dataset = pd.read_csv(r"C:\Users\vipul\Downloads\Dataset.csv", encoding='latin1')
dataset

,Speech,Intent
0,send mail,Send Email
1,send email,Send Email
2,hey jarvis send mail,Send Email
3,mind to send a mail?,Send Email
4,email,Send Email
5,mind to send a email?,Send Email
6,compose a mail,Send Email
7,draft a mail,Send Email
8,please send a mail,Send Email
9,send a message,Send Email


In [8]:
X = dataset['Speech']
y = dataset['Intent']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
nlp = spacy.load('en_core_web_sm')

In [11]:
def preprocess_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])


In [12]:
X_train_processed = X_train.apply(preprocess_text)
X_test_processed = X_test.apply(preprocess_text)


In [13]:

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train_processed)
X_test_tfidf = vectorizer.transform(X_test_processed)


In [14]:
model = MultinomialNB()
model.fit(X_train_tfidf, y_train, sample_weight=None)

MultinomialNB()

In [15]:
y_pred = model.predict(X_test_tfidf)
print("Classification Report:\n")
print(classification_report(y_test, y_pred))

# Save the model and vectorizer
joblib.dump(model, 'jarvis_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

Classification Report:

               precision    recall  f1-score   support

  Create List       1.00      1.00      1.00         1
    Greetings       1.00      1.00      1.00         2
Search Google       1.00      1.00      1.00         3
   Send Email       1.00      1.00      1.00         3
 Set Reminder       1.00      1.00      1.00         2

     accuracy                           1.00        11
    macro avg       1.00      1.00      1.00        11
 weighted avg       1.00      1.00      1.00        11



['vectorizer.pkl']

In [16]:
weights = {
    "feature_log_prob": model.feature_log_prob_.tolist(),
    "class_log_prior": model.class_log_prior_.tolist(),
    "classes": model.classes_.tolist(),
}

In [17]:
with open("model_weights.json", "w") as weight_file:
    json.dump(weights, weight_file)


In [18]:
def cli():
    print("\nWelcome to J.A.R.V.I.S NLP Classifier!")
    print("Type your phrase to classify or 'exit' to quit.")

    # Load the saved model and vectorizer
    model = joblib.load('jarvis_model.pkl')
    vectorizer = joblib.load('vectorizer.pkl')

    while True:
        user_input = input("\nYour Input: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break

        # Transform the input and make a prediction
        input_tfidf = vectorizer.transform([user_input])
        prediction = model.predict(input_tfidf)
        print(f"Predicted Intent: {prediction[0]}")




In [19]:
if __name__ == "__main__":
    if len(sys.argv) > 1 and sys.argv[1] == 'cli':
        cli()

In [ ]:
cli()


Welcome to J.A.R.V.I.S NLP Classifier!
Type your phrase to classify or 'exit' to quit.



Your Input:  wikipedia


Predicted Intent: Greetings



Your Input:  reemind me of a player


Predicted Intent: Greetings



Your Input:  remind me of a player


Predicted Intent: Set Reminder
